In [1]:
import os, sys
import pandas as pd

#import the functions
module_path = os.path.abspath(os.path.join('../src/'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import rtcw_functions as RTCW

db_name = 'rtcw_db.db'
demos_folder = 'demos'

cwd = os.getcwd()
demos_path = os.path.join(cwd, '../', demos_folder)

exe_name = 'Anders.Gaming.LibTech3.exe'
demos_folder = 'demos'
root_path = 'C:\\Users\\Jelle\\Documents\\GIT\\rtcw_demo_analyzer' #for executable we need a direct path

In [2]:
#get demos_dct
demos_dct = RTCW.make_dictionary(demos_path)

parsed 100 matches
parsed 200 matches
parsed 300 matches
parsed 400 matches
finished parsing all matches!


In [3]:
#load obituaries in a df
import sqlite3_connector
reload(sqlite3_connector)
db = sqlite3_connector.sqlite3_connector(os.path.join(root_path, db_name))

In [12]:
obituary_df = db.get_table_as_df(table_name= 'obituary')
player_df = db.get_table_as_df(table_name= 'player')
obituary_df = RTCW.add_match_data(obituary_df, player_df, demos_dct)
obituary_df = obituary_df.loc[obituary_df['League'] == 'qcon02']
obituary_df = obituary_df.loc[obituary_df['Importance'] == 'wb-round1']

obituary table fetched in 0.42 seconds
player table fetched in 0.04 seconds


In [16]:
df_spree = RTCW.get_kill_sprees(obituary_df, 
                                maxtime_secs = 10, 
                                include_weapon_filter = None,
                                exclude_weapon_filter = None, 
                                minspree = 3,
                                verbose = True)

all done!


In [17]:
df_spree[['match', 'demo', 'player', 'spreecount', 'weapons']]

,match,demo,player,spreecount,weapons
0,rtcw_2002.08.15_qcon02_ocrana_vs_diy_wb-round1,01_mp_base-OCRANAvsDiY_QCon1.dm_59,Ocrlove_D-link,4,5-7-7-7-
1,rtcw_2002.08.15_qcon02_ocrana_vs_diy_wb-round1,01_mp_base-OCRANAvsDiY_QCon1.dm_59,Ocrlove_D-link,3,7-7-15-
2,rtcw_2002.08.15_qcon02_ocrana_vs_diy_wb-round1,01_mp_base-OCRANAvsDiY_QCon1.dm_59,Ocrrep_D-link,3,17-17-17-
3,rtcw_2002.08.15_qcon02_ocrana_vs_diy_wb-round1,02_mp_base-OCRANAcsDiY_Qcon2.dm_59,Ocrlove_D-link,3,7-7-7-
4,rtcw_2002.08.15_qcon02_ocrana_vs_diy_wb-round1,02_mp_base-OCRANAcsDiY_Qcon2.dm_59,Diy.toolboy,4,6-7-7-7-
5,rtcw_2002.08.15_qcon02_ocrana_vs_diy_wb-round1,02_mp_base-OCRANAcsDiY_Qcon2.dm_59,Ocrdeep_D-link,3,5-5-5-
6,rtcw_2002.08.15_qcon02_affliction_vs_wsw_wb-ro...,A_VS_WSW.dm_59,-a-lordbugzer,4,7-7-7-7-
7,rtcw_2002.08.15_qcon02_affliction_vs_wsw_wb-ro...,A_VS_WSW.dm_59,-a-lordbugzer,4,5-5-5-5-
8,rtcw_2002.08.15_qcon02_affliction_vs_wsw_wb-ro...,A_VS_WSW.dm_59,Wswexclusive,3,18-18-16-
9,rtcw_2002.08.15_qcon02_affliction_vs_wsw_wb-ro...,A_VS_WSW.dm_59,Wswexclusive,3,17-17-17-


In [18]:
RTCW.cut_demos(root_path, demos_dct, df_spree, offset_start = 8, offset_end = 8, transform_to_dm_60 = True)
RTCW.generate_capture_list(df_spree, transform_to_dm_60 = True)

In [ ]:
for demo in df_spree.md5:
    for k in demos_dct:
        if demo in demos_dct[k][0]:
            print k

In [ ]:
reload(RTCW)

In [ ]:
obituary_df

In [ ]:

for k in demos_dct:
    if 'mp_sub-222842-[CUP]Veracious.dm_59' in demos_dct[k][1]:
        print demos_dct[k]